In [2]:
import json
import math
import re

#import ckanapi
import geopandas as gpd
import nltk
import numpy as np
import pandas as pd
import requests

from nltk.corpus import wordnet
from shapely.geometry import shape
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from datetime import datetime as dt

nltk.download('wordnet')

/opt/conda/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
def score_usability(columns, data):
    '''
        How easy is it to use the data given how it is organized/structured?
        
        TODO's: 
            * level of nested fields?
            * long vs. wide?
            * if ID columns given, are these ID's common across datasets?
    '''
    
    def parse_col_name(s):
        camel_to_snake = re.sub(
            '([a-z0-9])([A-Z])', 
            r'\1_\2', 
            re.sub('(.)([A-Z][a-z]+)', r'\1_\2', s)
        ).lower()

        return camel_to_snake == s, [x for x in re.split('-|_|\s', camel_to_snake) if len(x)]

    metrics = {
        'col_names': 0, # Are the column names easy to understand?
        'col_constant': 1 # Are there columns where all values are constant?
    }
    
    for f in columns:
        is_camel, words = parse_col_name(f)
        eng_words = [ w for w in words if len(wordnet.synsets(w)) ]

        if len(eng_words) / len(words) > 0.8:
            metrics['col_names'] += (1 if not is_camel else 0.5) / len(columns)
        
        if not f == 'geometry' and data[f].nunique() <= 1:
            metrics['col_constant'] -= 1 / len(columns)
    
    if isinstance(data, gpd.GeoDataFrame):
        counts = data['geometry'].is_valid.value_counts()
        
        metrics['geo_validity'] = 1 - (counts[False] / (len(data) * 0.05)) if False in counts else 1
    
    return np.mean(list(metrics.values()))

In [4]:
data = pd.read_csv("/mnt/data/DOB_NOW__Certificate_of_Occupancy.csv")
#data.index = data["Index Title"]
data.head()

,JOB FILING NAME,JOB TYPE,BIN,BOROUGH,HOUSE NO,STREET NAME,BLOCK,LOT,ZIP CODE,SUBMITTED DATE,...,longitude,communityDistrict,communityDistrictBoroughCode,communityDistrictNumber,cityCouncilDistrict,censusTract2010,buildingIdentificationNumber,bbl,nta,ntaName
0,01,ALTERATION TYPE 1,1000003,MANHATTAN,10,SOUTH STREET,2.0,2.0,10004.0,01/25/2022 12:00:00 AM,...,-74.011631,101.0,1.0,1.0,1.0,9.0,1000003.0,1.000020e+09,MN25,Battery Park City-Lower Manhattan
1,01,ALTERATION TYPE 1,1000003,MANHATTAN,10,SOUTH STREET,2.0,2.0,10004.0,01/27/2022 12:00:00 AM,...,-74.011631,101.0,1.0,1.0,1.0,9.0,1000003.0,1.000020e+09,MN25,Battery Park City-Lower Manhattan
2,01,ALTERATION TYPE 1,1000003,MANHATTAN,10,SOUTH STREET,2.0,2.0,10004.0,05/03/2021 12:00:00 AM,...,-74.011631,101.0,1.0,1.0,1.0,9.0,1000003.0,1.000020e+09,MN25,Battery Park City-Lower Manhattan
3,01,ALTERATION TYPE 1,1000003,MANHATTAN,10,SOUTH STREET,2.0,2.0,10004.0,08/13/2021 12:00:00 AM,...,-74.011631,101.0,1.0,1.0,1.0,9.0,1000003.0,1.000020e+09,MN25,Battery Park City-Lower Manhattan
4,01,ALTERATION TYPE 1,1000003,MANHATTAN,10,SOUTH STREET,2.0,2.0,10004.0,11/16/2021 12:00:00 AM,...,-74.011631,101.0,1.0,1.0,1.0,9.0,1000003.0,1.000020e+09,MN25,Battery Park City-Lower Manhattan


In [5]:
cols = data.columns
cols
data[cols]

,JOB FILING NAME,JOB TYPE,BIN,BOROUGH,HOUSE NO,STREET NAME,BLOCK,LOT,ZIP CODE,SUBMITTED DATE,...,longitude,communityDistrict,communityDistrictBoroughCode,communityDistrictNumber,cityCouncilDistrict,censusTract2010,buildingIdentificationNumber,bbl,nta,ntaName
0,01,ALTERATION TYPE 1,1000003,MANHATTAN,10,SOUTH STREET,2.0,2.0,10004.0,01/25/2022 12:00:00 AM,...,-74.011631,101.0,1.0,1.0,1.0,9.0,1000003.0,1.000020e+09,MN25,Battery Park City-Lower Manhattan
1,01,ALTERATION TYPE 1,1000003,MANHATTAN,10,SOUTH STREET,2.0,2.0,10004.0,01/27/2022 12:00:00 AM,...,-74.011631,101.0,1.0,1.0,1.0,9.0,1000003.0,1.000020e+09,MN25,Battery Park City-Lower Manhattan
2,01,ALTERATION TYPE 1,1000003,MANHATTAN,10,SOUTH STREET,2.0,2.0,10004.0,05/03/2021 12:00:00 AM,...,-74.011631,101.0,1.0,1.0,1.0,9.0,1000003.0,1.000020e+09,MN25,Battery Park City-Lower Manhattan
3,01,ALTERATION TYPE 1,1000003,MANHATTAN,10,SOUTH STREET,2.0,2.0,10004.0,08/13/2021 12:00:00 AM,...,-74.011631,101.0,1.0,1.0,1.0,9.0,1000003.0,1.000020e+09,MN25,Battery Park City-Lower Manhattan
4,01,ALTERATION TYPE 1,1000003,MANHATTAN,10,SOUTH STREET,2.0,2.0,10004.0,11/16/2021 12:00:00 AM,...,-74.011631,101.0,1.0,1.0,1.0,9.0,1000003.0,1.000020e+09,MN25,Battery Park City-Lower Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14261,I1,New Building,5863165,STATEN ISLAND,1,EVENTS PLAZA,9999.0,1.0,10301.0,02/11/2022 12:00:00 AM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14262,I1,New Building,5863165,STATEN ISLAND,1,EVENTS PLAZA,9999.0,1.0,10301.0,02/25/2022 12:00:00 AM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14263,I1,New Building,5863165,STATEN ISLAND,1,EVENTS PLAZA,9999.0,1.0,10301.0,08/06/2021 12:00:00 AM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14264,I1,New Building,5863165,STATEN ISLAND,1,EVENTS PLAZA,9999.0,1.0,10301.0,12/10/2021 12:00:00 AM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#data['index_col'] = data.index
data.head()

,JOB FILING NAME,JOB TYPE,BIN,BOROUGH,HOUSE NO,STREET NAME,BLOCK,LOT,ZIP CODE,SUBMITTED DATE,...,longitude,communityDistrict,communityDistrictBoroughCode,communityDistrictNumber,cityCouncilDistrict,censusTract2010,buildingIdentificationNumber,bbl,nta,ntaName
0,01,ALTERATION TYPE 1,1000003,MANHATTAN,10,SOUTH STREET,2.0,2.0,10004.0,01/25/2022 12:00:00 AM,...,-74.011631,101.0,1.0,1.0,1.0,9.0,1000003.0,1.000020e+09,MN25,Battery Park City-Lower Manhattan
1,01,ALTERATION TYPE 1,1000003,MANHATTAN,10,SOUTH STREET,2.0,2.0,10004.0,01/27/2022 12:00:00 AM,...,-74.011631,101.0,1.0,1.0,1.0,9.0,1000003.0,1.000020e+09,MN25,Battery Park City-Lower Manhattan
2,01,ALTERATION TYPE 1,1000003,MANHATTAN,10,SOUTH STREET,2.0,2.0,10004.0,05/03/2021 12:00:00 AM,...,-74.011631,101.0,1.0,1.0,1.0,9.0,1000003.0,1.000020e+09,MN25,Battery Park City-Lower Manhattan
3,01,ALTERATION TYPE 1,1000003,MANHATTAN,10,SOUTH STREET,2.0,2.0,10004.0,08/13/2021 12:00:00 AM,...,-74.011631,101.0,1.0,1.0,1.0,9.0,1000003.0,1.000020e+09,MN25,Battery Park City-Lower Manhattan
4,01,ALTERATION TYPE 1,1000003,MANHATTAN,10,SOUTH STREET,2.0,2.0,10004.0,11/16/2021 12:00:00 AM,...,-74.011631,101.0,1.0,1.0,1.0,9.0,1000003.0,1.000020e+09,MN25,Battery Park City-Lower Manhattan


In [7]:
score_usability(cols, data)

0.8362068965517242

In [8]:
## NYPD MOS data
mos = pd.read_csv("/mnt/data/NYPD_OIP_Officer_MOS.csv")
#data.index = data["Index Title"]
mos.head()

,PROFILE_ID,RANK,COMMAND,APPOINTMENT_DATE,ARRESTS_TOTAL,DEPARTMENT_RECOGNITIONS,EXPORT_DATE
0,EFA48BB6-F2F7-4717-A8A5-5B0C37EA9F5E,POLICE OFFICER,107 PRECINCT,01/06/2016 12:00:00 AM,55,2,06/12/2022 12:00:00 AM
1,2DE128A9-EF16-4D06-AAA3-CCC5DF5E301E,DETECTIVE 3RD GRADE,062 DET SQUAD,04/15/1997 12:00:00 AM,758,3,06/12/2022 12:00:00 AM
2,26C5E7DC-A575-402D-B232-D9FADC267E6A,POLICE OFFICER,049 PRECINCT,10/07/2019 12:00:00 AM,25,0,06/12/2022 12:00:00 AM
3,EABFF759-433D-458C-92FB-CCD1AE994296,POLICE OFFICER,123 PRECINCT,07/11/2005 12:00:00 AM,111,0,06/12/2022 12:00:00 AM
4,973DB185-30D4-498D-BB57-9213A7976621,DETECTIVE SPECIALIST,HOUSING PSA 6,01/09/2012 12:00:00 AM,318,4,06/12/2022 12:00:00 AM


In [9]:
mos_cols = mos.columns
mos_cols

Index(['PROFILE_ID', 'RANK', 'COMMAND', 'APPOINTMENT_DATE', 'ARRESTS_TOTAL',
       'DEPARTMENT_RECOGNITIONS', 'EXPORT_DATE'],
      dtype='object')

In [10]:
score_usability(mos_cols, mos)

0.9285714285714285

In [11]:
## dob facades compliane - BAD dataset test
bad = pd.read_csv("/mnt/data/DOB_NOW__Safety___Facades_Compliance_Filings.csv")
bad.head()

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3134: DtypeWarning: Columns (10,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,TR6_NO,CONTROL_NO,FILING_TYPE,CYCLE,BIN,HOUSE_NO,STREET_NAME,BOROUGH,BLOCK,LOT,...,FILING_DATE,FILING_STATUS,PRIOR_CYCLE_FILING_DATE,PRIOR_STATUS,FIELD_INSPECTION_COMPLETED_DATE,QEWI_SIGNED_DATE,LATE_FILING_AMT,FAILURE_TO_FILE_AMT,FAILURE_TO_COLLECT_AMT,COMMENTS
0,TR6-913448-9A-N1,913448,Auto-Generated,9,4114712.0,143-45,SANFORD AVENUE,QUEENS,5049,38,...,NaN,No Report Filed,NaN,NaN,NaN,NaN,11750.0,1000.0,0.0,NaN
1,TR6-913451-9A-N1,913451,Auto-Generated,9,3393807.0,15,OLIVER STREET,BROOKLYN,6099,1,...,NaN,No Report Filed,NaN,NaN,NaN,NaN,0.0,0.0,63400.0,NaN
2,TR6-913456-9A-N1,913456,Auto-Generated,9,1077623.0,180,ELDRIDGE STREET,MANHATTAN,415,12,...,NaN,No Report Filed,NaN,NaN,NaN,NaN,4250.0,0.0,0.0,NaN
3,TR6-913458-9A-N1,913458,Auto-Generated,9,4001141.0,41-46,50 STREET,QUEENS,134,1,...,NaN,No Report Filed,NaN,NaN,NaN,NaN,13250.0,2000.0,1000.0,NaN
4,TR6-913460-9A-N1,913460,Auto-Generated,9,1088779.0,220,EAST 19 STREET,MANHATTAN,899,46,...,NaN,No Report Filed,NaN,NaN,NaN,NaN,500.0,0.0,0.0,PHILIP DEANS - OWNER - PHN# 212-673-6262EMAIL:...


In [12]:
bad_cols = bad.columns
bad_cols

Index(['TR6_NO', 'CONTROL_NO', 'FILING_TYPE', 'CYCLE', 'BIN', 'HOUSE_NO',
       'STREET_NAME', 'BOROUGH', 'BLOCK', 'LOT', 'SEQUENCE_NO', 'SUBMITTED_ON',
       'CURRENT_STATUS', 'QEWI_NAME', 'QEWI_BUS_NAME', 'QEWI_BUS_STREET_NAME',
       'QEWI_CITY', 'QEWI_STATE', 'QEWI_ZIP', 'QEWI_NYS_LIC_NO', 'OWNER_NAME',
       'OWNER_BUS_NAME', 'OWNER_BUS_STREET_NAME', 'OWNER_CITY', 'OWNER_ZIP',
       'OWNER_STATE', 'FILING_DATE', 'FILING_STATUS',
       'PRIOR_CYCLE_FILING_DATE', 'PRIOR_STATUS',
       'FIELD_INSPECTION_COMPLETED_DATE', 'QEWI_SIGNED_DATE',
       'LATE_FILING_AMT', 'FAILURE_TO_FILE_AMT', 'FAILURE_TO_COLLECT_AMT',
       'COMMENTS'],
      dtype='object')

In [13]:
score_usability(bad_cols, bad)

0.7777777777777779